#### `Workflow`

0. Import Important Libraries. 


### **`Import Important Libraries.`**

In [ ]:
# all the important liberires from pytorch
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter

In [ ]:
import math

In [ ]:
#HuggingFace linraries
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

In [ ]:
# pathlib
from pathlib import Path

In [ ]:
#typing
from typing import Any

In [ ]:
# Library for progress bars in loops
from tqdm import tqdm

In [ ]:
# Importing Library of warnings
import warnings

## 1. `Input Embedding.`

`d_model`: 512

In Input Embedding, we multiply those weights, by `np.sqrt(d_model)`

Example Sentence:

`English sentence`: The animal didn't cross the street because it was too tired. 

**1. `Tokens`**
- Converting sentence into a single words. 

['The', 'animal', 'didn't', 'cross', 'the', 'street', 'because', 'it', 'was', 'too', 'tired' ]

In [ ]:
class InputEmbeddings(nn.Module):
    def __init__(self, d_model:int, vocab_size:int):
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        return self.embedding(x) * math.sqrt(self.d_model)

## 2. `Positional Embedding`

In [ ]:
class PositionalEncoding(nn.Module):
    '''
    max_sequence_length: Like a length of the token.
    '''
    def __init__(self, d_model: int, seq_len: int, dropout: float):
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(seq_len, d_model)
        position = torch.arange(0, seq_len, dtype=float).unsqueeze(1) # [seq_len, 1]
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)
    
    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False)
        return self.dropout(x)

## 3. `Multihead Attention (Self-Head Attention)`